## Gufhtugu Publication Dataset Challenge

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 2.csv",encoding='cp1252')
#reference https://stackoverflow.com/a/45530363

In [ ]:
data.head(10)

### As there is some values are not displaying correctly so try to use othe csv file

In [ ]:
data = pd.read_csv("/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 4.csv")

In [ ]:
#no of rows and number of columns
data.shape

### there are 19187 rows and 5 columns in dataset

In [ ]:
data.head(10)

In [ ]:
#no of rows and number of columns
data.shape

In [ ]:
df=data

## Expolatory Data Analysis


In [ ]:
# Number of rows and columns in the Data set
df.shape

In [ ]:
# check the rows have null values in dataset
df[pd.isnull(df).any(axis=1)]

In [ ]:
# the number of unique Order Numbers
df['Order Number'].nunique()

I was expecting to see the 19187 unique row values for Order number as there is not null value in the Order Number so check for duplicates

In [ ]:
#df.duplicated(subset=['Order Number']).nunique()
# Select all duplicate rows based on OrderNumber column
duplicateRowsDF = df[df.duplicated(['Order Number'])]
duplicateRowsDF

In [ ]:
# There are duplicate rows in the dataset
duplicate=df.loc[(df['Order Number']==65878) | (df['Order Number']==55816)]
duplicate

### Remove the duplicate from the dataset
there are duplicate rows with same order number and details


In [ ]:
df.drop_duplicates(subset=['Order Number'], keep= 'first',inplace=True) 
df.shape

In [ ]:
#there are null values but we are not deleting those rows
data2=df

### Check the number of completed, returned and canceled Orders

In [ ]:
df['Order Status'].unique()

In [ ]:
print(df['Order Status'].value_counts())
#print(df.groupby('Order Status').agg(['count']))

In [ ]:
# count plot of Order Status
import seaborn as sns
import matplotlib.pyplot as plt
df = data
sns.catplot(x="Order Status", kind="count", data=df)
plt.show()

## Percentage of Completed,Returned and Canceled Orders

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize=(8,5))
total = float(len(df))
ax =sns.countplot(x="Order Status", hue="Order Status", data=df)
plt.title('Order Status', fontsize=20)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:.1f}%'.format(100 * height/total),
            ha="center") 
plt.show()

In [ ]:
# Making a list of missing value types
missing_values = ["n/a", "na", "??????? ?? ???? ??????"]
def tidy_split(df, column, sep='/', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df
#source https://stackoverflow.com/a/39946744

In [ ]:
dataclean=tidy_split(df, 'Book Name')
dataclean.head(10)

In [ ]:
#sns.catplot(y="Book Name", kind="count", data=dataclean2)
#bookdata=pd.DataFrame(data2clean['Book Name'].value_counts())
bookdata = dataclean['Book Name'].value_counts().rename_axis('Book_Name').reset_index(name='counts')
bookdata


In [ ]:
dataclean['Book Name'].nunique()

### There are 1778 titles are Ordered


In [ ]:
#20 Most sold books
bookdata.head(20)


In [ ]:
print(bookdata.columns)
plotdata=bookdata.head(10)
df2 = plotdata.set_axis(['Book Name', 'Number Sold'], axis=1, inplace=False)

In [ ]:
# code adapted from: https://plotly.com/python/bar-charts/
import plotly.express as px

fig = px.bar(df2, x="Book Name", y="Number Sold", title="Top 10 Books")
fig.show()

### Returned Books

In [ ]:
# list of returned books
returned=dataclean[dataclean['Order Status']=="Returned"]['Book Name'].value_counts().rename_axis('Book_Name').reset_index(name='counts')
returned

In [ ]:
# list of canceled books
canceled=dataclean[dataclean['Order Status']=="Canceled"]['Book Name'].value_counts().rename_axis('Book_Name').reset_index(name='counts')
canceled

## Now look at the City data

In [ ]:
# look at the city
df.columns

In [ ]:
#top 10 cities from where got orders
df['City (Billing)'].value_counts().head(10)

In [ ]:
# look at the Order Date  to see the distribution  
dataclean.dtypes


In [ ]:
import datetime as dt
#Thanks to @hussainsaddam12 for this codeblock idea
df=dataclean
df["Order Date"] = pd.DatetimeIndex(df["Order Date"])
df['Date'] = df['Order Date'].dt.date
df['Time'] = df['Order Date'].dt.time
df["Day"] = df["Order Date"].dt.day_name()
df["Month"] = df["Order Date"].dt.month_name()
df['Year'] = df["Order Date"].dt.year
df.shape


In [ ]:
df.reset_index(drop=True)

## Tasks to Complete
- What is the best-selling book?
- Visualize order status frequency
- Find a correlation between date and time with order status
- Find a correlation between city and order status
- Find any hidden patterns that are counter-intuitive for a layman
- Can we predict number of orders, or book names in advance?

In [ ]:
# Best Selling Book
complete=df[df['Order Status']=="Completed"]['Book Name'].value_counts().rename_axis('Book_Name').reset_index(name='counts')
bestbook=complete.head(5)
fig = px.bar(bestbook, x="Book_Name", y='counts', title="Top 5 Books")
fig.show()

### Order Status Frequency

In [ ]:
# the cithy from where we got the most orders with completed status
#i am using the given data set excluding  duplicate order Numbers rows
# as there are more than  one books were in orders and we are looking at orders so used the data without spliting
# the book Name column data

completeorder=data2[data2['Order Status']=="Completed"]['City (Billing)'].value_counts().rename_axis('City').reset_index(name='counts')
order=completeorder.head(10)
#fig = px.bar(order, x="City", y="counts", title="Top 10 Cities")
import plotly.graph_objects as go
x=order["City"]
y=order["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])

fig.show()
#fig.show()
order.transpose()

In [ ]:
data2.groupby('Order Status')['City (Billing)'].count()

In [ ]:
returnedorder=data2[data2['Order Status']=="Returned"]['City (Billing)'].value_counts().rename_axis('City').reset_index(name='counts')
r_order=returnedorder.head(10)
import plotly.graph_objects as go
x=r_order["City"]
y=r_order["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Returned Orders')
fig.show()
#fig.show()
r_order.transpose()

In [ ]:
# Canceld Orders
cancelorder=data2[data2['Order Status']=="Canceled"]['City (Billing)'].value_counts().rename_axis('City').reset_index(name='counts')
c_order=cancelorder.head(10)
import plotly.graph_objects as go
x=c_order["City"]
y=c_order["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Canceld Orders')
fig.show()
#fig.show()
c_order.transpose()

In [ ]:
# these are number of books ordered Frequency By Months
ordermonth=df['Month'].value_counts().rename_axis('Month').reset_index(name='counts')
  
#ordermonth
x=ordermonth["Month"]
y=ordermonth["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Monthly Orders (No of Books)')
fig.show()
#fig.show()
ordermonth.transpose()

In [ ]:
# to get the number of orders
data2["Order Date"] = pd.DatetimeIndex(data2["Order Date"])
data2["Month"] = data2["Order Date"].dt.month_name()
#df['Year'] = df["Order Date"].dt.year
ordermonth=data2['Month'].value_counts().rename_axis('Month').reset_index(name='counts')
  
#ordermonth
x=ordermonth["Month"]
y=ordermonth["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Monthly Orders')
fig.show()
#fig.show()
ordermonth.transpose()

In [ ]:
data2["Day"] = data2["Order Date"].dt.day_name()
orderday=data2['Day'].value_counts().rename_axis('Day').reset_index(name='counts')
  
#ordermonth
x=orderday["Day"]
y=orderday["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Orders Day')
fig.show()
#fig.show()
orderday.transpose()

## New data file is added look at that

In [ ]:
data = pd.read_csv("/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv")

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head(10)

In [ ]:
df=data[data['Order Number']==3162]['Book Name']

In [ ]:
pd.set_option('display.max_colwidth', None)
df

## Payment Methods

In [ ]:
data['Payment Method'].unique()

In [ ]:
# check the rows have null values in dataset
data[pd.isnull(data).any(axis=1)]

### there are 10 rows where payment method is NaN

In [ ]:
data.head(10)

In [ ]:
data['Payment Method'].value_counts()

### Change the cash on delivery (COD) to cash on delivery 

In [ ]:
#change the cash on delivery (COD) to Cash on delivery
data.loc[data['Payment Method']=='Cash on Delivery (COD)' , 'Payment Method']= 'Cash on delivery'

In [ ]:
data['Payment Method'].value_counts()

In [ ]:
# Payment Method
paymentmethod=data['Payment Method'].value_counts().rename_axis('Payment Method').reset_index(name='counts')
x=paymentmethod["Payment Method"]
y=paymentmethod["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Payment Methods')
fig.show()

## As it is obvious from the above plot that cash on delivery is popular method of payment


## Order Status

In [ ]:
data['Order Status'].value_counts()

In [ ]:
# Order status
orderstatus=data['Order Status'].value_counts().rename_axis('Order Status').reset_index(name='counts')
x=orderstatus["Order Status"]
y=orderstatus["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Order Status')
fig.show()

In [ ]:
data['Book Name'].head(10)

In [ ]:
#new = data["Book Name"].str.split("/",expand = True) 

In [ ]:
#split the Book Name column values
cleandata=tidy_split(data, 'Book Name')


In [ ]:
cleandata.shape

In [ ]:
cleandata.reset_index(drop=True)
cleandata.head(20)

### there are two issues one is with (C++) ++سی/سی as there is / is separating them so rather than One row we are getting two rows for this one book. when spliting the data
### there are books where more than 1 copy is  orderd as e.g.order Number 307

In [ ]:
data['Book Name'] = data['Book Name'].str.replace(r"\(C\++\)\s\++سی/سی" , '(C++) ++سی-سی')
#split the Book Name column values
cleandata=tidy_split(data, 'Book Name')
cleandata.head(20)


In [ ]:
df=cleandata
complete=df[df['Order Status']=="Completed"]['Book Name'].value_counts().rename_axis('Book_Name').reset_index(name='counts')
bestbook=complete.head(10)
fig = px.bar(bestbook, x="Book_Name", y='counts', title="Top 10 Books")
fig.show()

In [ ]:
bestbook

### As look at the City column there are inconsistency 
like faisalabad in some rows and FSD in some, same for dera Ismail Khan  in a row also entered as d i khan so need to fix it


In [ ]:
# removing null values to avoid errors   
data.dropna(inplace = True)   
  
# Applying upper() method on 'City' column 
data['City']=data['City'].apply(lambda x: x.upper())

In [ ]:
data.loc[data['City']=='FSD' , 'City']= 'FAISALABAD'
data.loc[data['City']=='D I KHAN' , 'City']= 'DERA ISMAIL KHAN'
data.loc[data['City']=='DEAR ISMAIL KHAN' , 'City']= 'DERA ISMAIL KHAN'

data.loc[data['City']=='KARACH' , 'City']= 'KARACHI'
data.loc[data['City']=='SAIHWAL', 'City']= 'SAHIWAL'
data.loc[data['City']=='ISB', 'City']= 'ISLAMABAD'
data.loc[data['City']=='LHR', 'City']= 'LAHORE'
data.loc[data['City']=='SARGOSHA', 'City']= 'SARGODHA'
data.loc[data['City']=='PESHAWAR KPK', 'City']= 'PESHAWAR'
data.loc[data['City']=='KPK', 'City']= 'PESHAWAR'
data.loc[data['City']=='RAIWIND LAHORE', 'City']= 'RAIWIND'
data.loc[data['City']=='قصور', 'City']='KASUR'
data.loc[data['City']=='KASOOR', 'City']='KASUR'
data.loc[data['City']=='RAHIMYARKHAN','City']='RAHIMYAR KHAN'


In [ ]:
data.loc[data['City']=='RAHIMYARKHAN']

In [ ]:
data['City'].nunique()

In [ ]:
data['City'].head(20)

In [ ]:
city=data['City'].value_counts().rename_axis('City').reset_index(name='counts')
city.tail(20)

In [ ]:
data['City'].nunique()

In [ ]:
completeorder=data[data['Order Status']=="Completed"]['City'].value_counts().rename_axis('City').reset_index(name='counts')
order=completeorder.head(10)
#fig = px.bar(order, x="City", y="counts", title="Top 10 Cities")
import plotly.graph_objects as go
x=order["City"]
y=order["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])

fig.show()

In [ ]:
completeorder

In [ ]:
data["Order Date & Time"] = pd.DatetimeIndex(data["Order Date & Time"])
data["Year"] = data["Order Date & Time"].dt.year
#df['Year'] = df["Order Date"].dt.year
orderyear=data['Year'].value_counts().rename_axis('Year').reset_index(name='counts')
  
#ordermonth
x=orderyear["Year"]
y=orderyear["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Yearly Orders')
fig.show()

## Order/year

In [ ]:
orderyear

In [ ]:
# Create a circle for the center of the plot
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(orderyear["counts"], labels=orderyear["Year"], colors=['skyblue','green','orange'],wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' })
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()


## Orders/ Month

In [ ]:
data["Order Date & Time"] = pd.DatetimeIndex(data["Order Date & Time"])
data["Month"] = data["Order Date & Time"].dt.month_name()
#df['Year'] = df["Order Date"].dt.year
ordermonth=data['Month'].value_counts().rename_axis('Month').reset_index(name='counts')
  
#ordermonth
x=ordermonth["Month"]
y=ordermonth["counts"]
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(x=x,
                             y=y,
                             text=y,
            textposition='auto',
        )])
fig.update_layout(title_text='Monthly Orders')
fig.show()

## Orders/week days

In [ ]:
data['Date'] = data["Order Date & Time"].dt.date
data['Time'] = data["Order Date & Time"].dt.time
data["Day"] = data["Order Date & Time"].dt.day_name()
plt.figure(figsize=(20, 8))
plt.title('Number of books Orders/Week Days')
plot = sns.countplot(x=data["Day"],data=data, hue="Order Status")
for p in plot.patches:
        plot.annotate(format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points', rotation=0)
plot.set_xticklabels(plot.get_xticklabels(), ha="center")
plt.show()